# OpenAI 매개변수

# 개요
OpenAI 모델에 요청을 보낼 때 여러 매개변수를 사용하여 모델의 동작과 출력을 제어할 수 있습니다. \
이러한 매개변수를 이해하면 텍스트 생성, 질문 응답 또는 기타 사용 사례에 맞게 응답을 세부 조정할 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
SEED = 123

# 매개변수: max_tokens
**설명**: 생성할 응답의 최대 토큰 수를 설정합니다. \
**기본값**: 16 \
**예제**: max_tokens=50

프롬프트의 토큰 수와 max_tokens의 합은 모델의 컨텍스트 길이를 초과할 수 없습니다. \
gpt-4o-mini의 경우 16,384 tokens이며 모델에 따라 다릅니다. 

In [10]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

tokens = [32, 64, 120, 200]
for token in tokens:
    print(f"Max Tokens: {token}\n")
    print(call_openai_with_max_tokens(token))
    print("\n" + "-"*80 + "\n")

Max Tokens: 32

최고의 반려동물을 선택하는 것은 개인적인 취향과 생활 방식에 따라 다릅니다. 다양한 반려동물 중에서 자신의 라이

--------------------------------------------------------------------------------

Max Tokens: 64

최고의 반려동물을 선택하는 것은 개인적인 취향과 생활 방식에 따라 다릅니다. 다양한 반려동물 중에서 자신의 라이

--------------------------------------------------------------------------------

Max Tokens: 64

최고의 반려동물은 개인의 라이프스타일, 취향, 그리고 환경에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물을 소개하자면:

1. **강아지**: 충성스럽고 사교적이며 다양한 품종이 있어 활동적인 사람

--------------------------------------------------------------------------------

Max Tokens: 120

최고의 반려동물은 개인의 라이프스타일, 취향, 그리고 환경에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물을 소개하자면:

1. **강아지**: 충성스럽고 사교적이며 다양한 품종이 있어 활동적인 사람

--------------------------------------------------------------------------------

Max Tokens: 120

최고의 반려동물은 개인의 라이프스타일, 성격, 생활 환경에 따라 다를 수 있습니다. 반려동물을 선택할 때 고려해야 할 몇 가지 요소는 다음과 같습니다:

1. **시간과 노력**: 일부 반려동물은 많은 시간과 노력이 필요합니다. 예를 들어, 개는 규칙적인 산책과 훈련이 필요하지만, 고양이는 덜 필요할 수 있습니다.

2. **공간**: 반려동물이 살아갈 공간의 크기도 고려해야 합니다. 대형견

------

# 매개변수: temperature

**설명**: 출력의 무작위성을 제어합니다. 낮은 값은 출력을 더 결정론적으로 만들고, 높은 값은 무작위성을 증가시킵니다. \
**값 범위**: 0에서 1 \
**기본값**: 1 \
**예제**: temperature=0.7

높은 값은 모델이 더 창의적인 출력을 생성하도록 합니다. \
창의적인 응용 프로그램에는 0.9를, 명확한 답변이 필요한 경우에는 0(최대 확률 샘플링)을 시도하세요.

---
**참고**: 일반적으로 이 매개변수 또는 top_p를 조정하되 둘 다 동시에 조정하지 않는 것을 권장합니다.


In [4]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [7]:
# Without seed and temperature, the response is different each time
call_openai(10, '최고의 반려동물은 ')

반려동물을 선택할 때 "최고의" 반려동물은 개인의 생활 방식, 취향, 필요에 따라 달라질 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 살펴보면 다음과 같습니다:

1. **강아지
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물 옵션과 각각의 장점을 소개하겠습니다:

1. **개**: 사회적이고 충성스러운 동물로, 많은
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물 옵션과 각각의 장점을 소개하겠습니다:

1. **개**: 사회적이고 충성스러운 동물로, 많은
최고의 반려동물은 개인의 생활 방식, 선호도, 필요에 따라 다릅니다. 어떤 사람들에게는 개가 최고의 반려동물이 될 수 있으며, 이는 그들이 동반자 관계와 활동적인 라이프 스타일을 즐기기 때문입니다. 다른
최고의 반려동물은 개인의 생활 방식, 선호도, 필요에 따라 다릅니다. 어떤 사람들에게는 개가 최고의 반려동물이 될 수 있으며, 이는 그들이 동반자 관계와 활동적인 라이프 스타일을 즐기기 때문입니다. 다른
최고의 반려동물은 개인의 생활 방식, 선호도, 필요에 따라 다릅니다. 다양한 반려동물 각각의 장점을 고려하여 선택하는 것이 중요합니다. 몇 가지 인기 있는 반려동물의 특징을 소개하겠습니다:

1. **개
최고의 반려동물은 개인의 생활 방식, 선호도, 필요에 따라 다릅니다. 다양한 반려동물 각각의 장점을 고려하여 선택하는 것이 중요합니다. 몇 가지 인기 있는 반려동물의 특징을 소개하겠습니다:

1. **개
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물 옵션을 고려해 보겠습니다:

1. **개**: 개는 충성스럽고 사회적이며 다양한 품종이 있어 각
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물 옵션을 고려해 보겠습니다:

1. **개**: 개는 충성스럽고 사회적이며 다양한

In [8]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, '최고의 반려동물은 ', temperature = 0, use_seed=True)

최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개해드리겠습니다.

1. **강아지**: 강아지는 충성스럽고 사회적이며
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개해드리겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개해드리겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기
최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물과 그 특징을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 품종이 있어 크기

# 매개변수: n
**설명**: 각 프롬프트에 대해 생성할 응답의 수를 지정합니다. \
**기본값**: 1 \
**예제**: n = 3 

---
**참고**: 이 매개변수는 여러 응답을 생성하므로 토큰 할당량을 빠르게 소모할 수 있습니다. 신중하게 사용하고 max_tokens 및 stop 설정이 적절한지 확인하세요.

In [15]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "최고의 반려동물은"}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)
    print("\n" + "-"*80 + "\n")

0 최고의 반려동물은 개인의 생활 방식, 취향, 그리고 필요에 따라 다를 수 있기 때문에 명확히 하나로 정의하기 어렵습니다. 몇 가지 예를 들어보면:

1. **강아지**: 충성심이 강하고 사람과 잘

--------------------------------------------------------------------------------

1 최고의 반려동물은 개인의 라이프스타일, 선호도, 생활 환경 등에 따라 다를 수 있습니다. 몇 가지 인기 있는 반려동물을 소개하겠습니다:

1. **개**: 충성스럽고 사회적이며 다양한 크기와 품종이

--------------------------------------------------------------------------------



# 매개변수: presence_penalty
**설명**: 텍스트에 이미 나타난 토큰을 기반으로 새 토큰에 페널티를 부여하여 모델이 새로운 토큰을 사용하도록 유도합니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: presence_penalty=0.5

In [ ]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=60,
                    presence_penalty=presence_penalty, 
                    
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

최고의 반려동물은 개인의 취향, 생활 방식, 환경 등에 따라 달라집니다. 각각의 반려동물은 고유한 장점과 요구 사항이 있습니다. 몇 가지 인기 있는 반려동물과 그들의 특징을 소개해드리겠습니다

--------------------------------------------------------------------------------

Presence Penalty: 0.5

최고의 반려동물은 개인의 취향, 생활 방식, 환경 등에 따라 달라집니다. 각각의 반려동물은 고유한 장점과 요구 사항이 있습니다. 몇 가지 인기 있는 반려동물과 그들의 특징을 소개해드리겠습니다

--------------------------------------------------------------------------------

Presence Penalty: 0.5

최고의 반려동물은 개인의 취향과 생활 방식에 따라 다릅니다. 어떤 사람에게는 개가 최고의 동반자가 될 수 있고, 다른 사람에게는 고양이가 더 맞을 수도 있습니다. 몇 가지 일반적인 반려동물 옵션과 각각의

--------------------------------------------------------------------------------

Presence Penalty: 1.0

최고의 반려동물은 개인의 취향과 생활 방식에 따라 다릅니다. 어떤 사람에게는 개가 최고의 동반자가 될 수 있고, 다른 사람에게는 고양이가 더 맞을 수도 있습니다. 몇 가지 일반적인 반려동물 옵션과 각각의

--------------------------------------------------------------------------------

Presence Penalty: 1.0

최고의 반려동물은 개인의 생활 방식, 취향, 필요에 따라 다를 수 있습니다. 다양한 종류의 반려동물이 있으며 각각 고유한 특성과 요구사항이 있습니다. 몇 가지 인기 있는 반려동물을 살펴보면:

1. **

# 매개변수: frequency_penalty
**설명**: 텍스트에 이미 나타난 빈도를 기반으로 새 토큰에 페널티를 부여하여 동일한 줄을 반복할 가능성을 줄입니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: frequency_penalty=0.5

#### 탐색할 사용 사례
1. **응답 비교** \
여러 응답을 생성하여 사용 사례에 가장 적합한 응답을 선택하세요.

2. **다양성 증가** \
다양한 응답을 얻기 위해 여러 응답을 생성하세요. 이는 창의적인 응용 프로그램에 유용합니다.

3. **강건성 향상** \
여러 응답을 생성하여 일관성과 정확성을 보장하세요.

#### 모범 사례
1. **프롬프트 길이 최적화** \
프롬프트를 간결하지만 정보가 충분하도록 유지하여 모델이 충분한 컨텍스트를 가지도록 하세요.

2. **Temperature 및 Top_p 조정** \
결정론적 응답과 창의적 응답 간의 균형을 맞추기 위해 이 매개변수를 사용하세요.

3. **토큰 사용량 모니터링** \
max_tokens 매개변수를 신중히 설정하여 비용과 응답 길이를 관리하세요.

4. **중지 시퀀스 사용** \
모델이 텍스트 생성을 중지해야 할 위치를 정의하여 원하는 컨텍스트 내에서 출력을 제어하세요.

5. **여러 응답 생성** \
n 매개변수를 사용하여 여러 응답을 생성하고 필요에 가장 적합한 응답을 선택하세요.